In [2]:
import numpy as np

In [5]:
c_p = 4200  # J/(kg K)

In [4]:
UA     = [   10,     5,    10] # W/K
C      = [40000, 40000, 40000] # J/(kg K)
dot_mi = [    0,     0,     0] # kg/s
dot_mo = [    0,     0,     0] # kg/s
T_inf  = 293.15  # k

N = 3 # number of layers

In [7]:
# mass balance
dot_m = np.zeros((N, N))
# dot_m[ein, aus]

In [ ]:
A = np.zeros((N, N))

A[0, 0]     = (-UA[0] - dot_mo[0]*c_p - dot_m[0, 1]*c_p)/C[0]
A[0, 1]     = (dot_m[1, 0]*c_p)/C[0]

for n in range(1, N-1):
    A[n, n-1] = (dot_m[n-1, n]*c_p)/C[n]
    A[n, n]   = (-UA[n] - dot_mo[n]*c_p - dot_m[n, n+1]*c_p - dot_m[n, n-1]*c_p)/C[n]
    A[n, n+1] = (dot_m[n+1, n]*c_p)/C[n]

A[N-1, N-2] = (dot_m[N-2, N-1]*c_p)/C[N-1]
A[N-1, N-1] = (-UA[N-1] - dot_mo[N-1]*c_p - dot_m[N-1, N-2]*c_p)/C[N-1]

In [9]:
A

array([[-0.00025 ,  0.      ,  0.      ],
       [ 0.      , -0.000125,  0.      ],
       [ 0.      ,  0.      , -0.00025 ]])